In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

Mounted at /content/gdrive
gdrive	sample_data


In [2]:
%cd gdrive/My\ Drive/project
!ls


/content/gdrive/My Drive/project
Backup		custom_model  IMAGES	  NewDataHammad  tools	   yolov3
custom_dataset	Demo.ipynb    model_data  __pycache__	 train.py


In [3]:
#!python tools/XML_to_YOLOv3.py
!nvidia-smi

Thu Dec 17 20:12:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import tensorflow as tf
print(tf.__version__)

2.4.0


In [5]:
from train import *
tf.keras.backend.clear_session()
main()

GPUs [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Model Weights Loaded.. Loading Dataset
400
epoch: 0 step:    2/400, lr:0.000000, giou_loss:   2.45, conf_loss: 426.05, prob_loss:   4.96, total_loss: 433.46
epoch: 0 step:    3/400, lr:0.000000, giou_loss:   1.83, conf_loss: 418.66, prob_loss:   3.84, total_loss: 424.33
epoch: 0 step:    4/400, lr:0.000000, giou_loss:   1.79, conf_loss: 419.70, prob_loss:   3.77, total_loss: 425.26
epoch: 0 step:    5/400, lr:0.000001, giou_loss:   2.88, conf_loss: 418.50, prob_loss:   5.19, total_loss: 426.57
epoch: 0 step:    6/400, lr:0.000001, giou_loss:   2.10, conf_loss: 417.40, prob_loss:   4.22, total_loss: 423.72
epoch: 0 step:    7/400, lr:0.000001, giou_loss:   2.93, conf_loss: 413.82, prob_loss:   5.51, total_loss: 422.26
epoch: 0 step:    8/400, lr:0.000001, giou_loss:   1.91, conf_loss: 417.65, prob_loss:   3.99, total_loss: 423.55
epoch: 0 step:    9/400, lr:0.000001, giou_loss:   1.97, conf_loss: 421.42, prob_loss: 

In [ ]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import glob
from google.colab.patches import cv2_imshow

In [ ]:
from yolov3.utils import load_yolo_weights, detect_image
from yolov3.configs import *
from yolov3.yolov3 import *

new_yolo = Create_Yolov3(input_size= 416, CLASSES=TRAIN_CLASSES,)
new_yolo.load_weights("custom_model/yolov3_custom_Tiny") # use keras weights

In [ ]:
input_shape = new_yolo.inputs[0].shape.as_list()
output_shape = new_yolo.outputs[0].shape.as_list()
input_shape[0] = 1
print(input_shape,output_shape)
export_path = "./Backup/main_tiny_new_yolo";
tf.saved_model.save(new_yolo,export_dir= export_path) 
func = tf.function(new_yolo).get_concrete_function(
    tf.TensorSpec(input_shape, new_yolo.inputs[0].dtype))
new_yolo.summary()

In [ ]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([func])
converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops=True
converter.experimental_new_converter =True
lite_model = converter.convert()

open("Backup/Keras_lite_model.tflite",'wb').write(lite_model)

In [ ]:
images = glob.glob("./IMAGES/test/*.jpg") 

for image in images:
    cv2.imwrite("./IMAGES/results/"+ image[image.rfind('/') + 1 :], img)